## Predicto: Dataset creation and model training

Notebook to create a dataset in order to train a lanquage model able to autocomplete tsx lines of code or css rules.

In [ ]:
## install cssbeautify-cli 
# ! npm install -g cssbeautify-cli

# !pip install transformers
# !pip install tokenizers
# !pip install PyGithub

In [1]:
from pathlib import Path

from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer
from tokenizers import ByteLevelBPETokenizer

from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer# loading tokenizer from the saved model path

import gc
import os
import shutil
import re
import json
import time
import random
import requests

import getpass

import pandas as pd

from github import Github

from torch.utils.tensorboard import SummaryWriter
import torch

In [2]:
## We need a list with all css properties

response = requests.get("https://gist.githubusercontent.com/davidhund/3bd6757d6a36a283b0a2933666bd1976/raw/4ccc47ed835c1be6bf0bfe00a32f427874da917c/all-css-properties.json")
data = response.json()
data = [x['property'] for x in data[2:]]
all_css_properties = []
for i in data:
    if not i in all_css_properties:
        all_css_properties.append(i)

## Options

In [12]:
## https://github.com/topics
# topic = 'css'
topic = 'typescript'

if topic == 'css':
    filetype = 'css'
elif topic == 'typescript':
    filetype = 'tsx'



EXPERIMENT = 'predicto_' + topic
experiment_path = "data/" + EXPERIMENT + "/"

path_test = 'data/predicto_' + topic + '_test.txt'
path_train = 'data/predicto_' + topic + '_train.txt'


if not os.path.exists(experiment_path):
    os.mkdir(experiment_path)

n_repo = 100 ## max 1000 repo to take from github

vocab_size=5000
n_positions=64

n_layer=8
n_head=8
n_emb=128
batch_size = 64
num_train_epochs = 1

## Helpers

In [4]:
def remove_comments(text):
    """ remove c-style comments. From https://www.saltycrane.com/blog/2007/11/remove-c-comments-python/
        text: blob of text with comments (can include newlines)
        returns: text with comments removed
    """
    pattern = r"""
                            ##  --------- COMMENT ---------
           /\*              ##  Start of /* ... */ comment
           [^*]*\*+         ##  Non-* followed by 1-or-more *'s
           (                ##
             [^/*][^*]*\*+  ##
           )*               ##  0-or-more things which don't start with /
                            ##    but do end with '*'
           /                ##  End of /* ... */ comment
         |                  ##  -OR-  various things which aren't comments:
           (                ## 
                            ##  ------ " ... " STRING ------
             "              ##  Start of " ... " string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^"\\]       ##  Non "\ characters
             )*             ##
             "              ##  End of " ... " string
           |                ##  -OR-
                            ##
                            ##  ------ ' ... ' STRING ------
             '              ##  Start of ' ... ' string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^'\\]       ##  Non '\ characters
             )*             ##
             '              ##  End of ' ... ' string
           |                ##  -OR-
                            ##
                            ##  ------ ANYTHING ELSE -------
             .              ##  Anything other char
             [^/"'\\]*      ##  Chars which doesn't start a comment, string
           )                ##    or escape
    """
    regex = re.compile(pattern, re.VERBOSE|re.MULTILINE|re.DOTALL)
    noncomments = [m.group(2) for m in regex.finditer(text) if m.group(2)]

    return "".join(noncomments)

## Get a few github repo based on the selected topic

In [5]:
## input here your github api key: https://github.com/settings/tokens
github_api_key = getpass.getpass() 

 ········································


In [6]:
g = Github(github_api_key)

repos = g.search_repositories(query=f'topic:{topic}',sort='stars')

it = 0
repos_clean = []
for repo in repos[:n_repo]:
    it += 1
    repos_clean.append(repo.full_name)
    if it % 50 == 0:
        time.sleep(1) ## we have to sleep a bit to give the api some rest

random.shuffle(repos_clean)
repos_clean_test = repos_clean[:int(len(repos_clean)/10)]
repos_clean_train = repos_clean[int(len(repos_clean)/10):]

## Clean and extract the files of interest

In [7]:
for repos_clean, path_data in zip([repos_clean_test, repos_clean_train],
                                                     [path_test, path_train]):

#     file_out = open(path_data, "w",)
    output_lines = []

    total_line_number = 0
    total_file_number = 0
    for repo in repos_clean:
        if repo == 'cdnjs/cdnjs': ## need to remove this one...
            continue
            
        folder = re.sub('/','-',repo)
        os.system('git clone  --depth 1 https://github.com/' + repo + ' data/' + folder)
        paths = []
        paths += [str(x) for x in Path("data/"+folder+'/').glob("**/*." + filetype)]
        line_number = 0
        for path in paths:
            path = path.replace(" ", "\\ ")
            if os.path.isfile(path):
                ## Remove comment
                with open(path, 'r', encoding="utf8", errors='ignore') as f:
                    code_w_comments = f.read()
                code_wo_comments = remove_comments(code_w_comments)
                fh = open(path, "w")
                fh.write(code_wo_comments)
                fh.close()
                if topic == 'css':
                    if os.path.exists('/tools/node/lib/node_modules/cssbeautify-cli/bin/cssbeautify-cli'):
                        os.system('/tools/node/lib/node_modules/cssbeautify-cli/bin/cssbeautify-cli -f '+
                                  path +' > ' + path)
                    else:
                        os.system(''~/node_modules/.bin/cssbeautify-cli -f '+
                                  path +' > ' + path)
                ## if the final file exist, append all lines to the list
                if os.path.isfile(path):
                    file_in = open(path, "r", errors='ignore')
                    lines = file_in.readlines()
                    for line in lines:
                        if (line != '\n'):                 
                            output_lines.append(line)
                            line_number += 1
                    file_in.close()      
        shutil.rmtree("data/"+folder)
        total_line_number +=  line_number
        total_file_number +=  len(paths)
    print('total_line_number: ' + str(total_line_number) + ' for ' + path_data)
    print('total_file_number: ' + str(total_file_number) + ' for ' + path_data)
    
    if topic == 'css':
        ## some more cleaning to get one css rules per line
        file_out = open(path_data, "w",)
        l = ''
        for line in output_lines:
            if (line != '}\n'):  
                line = re.sub('\n','',line)
                line = re.sub(' +',' ',line)
                l += line
            else:
                l += '}\n'
                if not l.startswith('@'):
                    file_out.write(l)
                l = ''
        file_out.close()
    else:
        file_out = open(path_data, "w",)
        for line in output_lines:
            file_out.write(line)
        file_out.close()

total_line_number: 62168 for data/predicto_typescript_test.txt
total_file_number: 531 for data/predicto_typescript_test.txt
total_line_number: 454185 for data/predicto_typescript_train.txt
total_file_number: 5443 for data/predicto_typescript_train.txt


## Tokenizer training

We need to remove long sample to avoid infering unfinished samples at test time, for this we first train a tokenizer and then applying it on our datasets.

In [8]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.pre_tokenizers import Whitespace

special_tokens=["<s>","<pad>","</s>","<unk>","<mask>",]
if topic == 'css':
    special_tokens += all_css_properties

tokenizer = ByteLevelBPETokenizer()
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train(files=[path_train],
                vocab_size=vocab_size,
                min_frequency=2,
                special_tokens=[
                    "<s>",
                    "<pad>",
                    "</s>",
                    "<unk>",
                    "<mask>",]
               ),
tokenizer.save_model(experiment_path)

['data/predicto_typescript/vocab.json', 'data/predicto_typescript/merges.txt']

In [9]:
for path in [path_train, path_test]:
    file_in = open(path,'r',)
    lines = file_in.readlines()    
    file_in.close()
    print(str(len(lines)) + ' originally lines in ' + path)
    file_out = open(path, "w",)
    for l in lines:
        if len(tokenizer.encode(l)) <= n_positions:
            file_out.write(l)
    file_out.close()

453752 originally lines in data/predicto_typescript_train.txt
62167 originally lines in data/predicto_typescript_test.txt


In [10]:
for path in [path_train, path_test]:
    file_in = open(path,'r',)
    lines = file_in.readlines()
    print(str(len(lines)) + ' after subset lines in ' + path)

453420 after subset lines in data/predicto_typescript_train.txt
61976 after subset lines in data/predicto_typescript_test.txt


## Model setup

In [13]:

tokenizer = GPT2Tokenizer.from_pretrained(experiment_path)

tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})# creating the configurations from which the model can be made
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_layer=n_layer,
    n_head=n_head,
    n_emb=n_emb,
    n_positions=n_positions
)# creating the model
model = GPT2LMHeadModel(config)
model.num_parameters()

60593664

## Data loader 

In [27]:
from transformers import LineByLineTextDataset


dataset_test = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_test,
    block_size=n_positions,
)

dataset_train = LineByLineTextDataset(
            tokenizer=tokenizer,
            file_path=path_train,
            block_size=n_positions,
        )

/opt/anaconda/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers-4.1.1-py3.8.egg/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py


In [28]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

## Training

In [29]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
            output_dir=experiment_path,
            overwrite_output_dir=True,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=batch_size,
            save_steps=10_000,
            save_total_limit=2,
        )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    # prediction_loss_only=True,
)

trainer.train()

trainer.evaluate()

Step,Training Loss
500,1.771101
1000,1.550462
1500,1.460529
2000,1.417736
2500,1.372226
3000,1.325910
3500,1.311637
4000,1.291306
4500,1.275830
5000,1.243019


{'eval_loss': 2.3431520462036133, 'epoch': 1.0}

## Save and reload the model and tokenizer 

In [32]:
trainer.save_model(experiment_path)

In [33]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re
import torch

tokenizer = GPT2Tokenizer.from_pretrained(experiment_path)
model = GPT2LMHeadModel.from_pretrained(experiment_path)

## Test the result

In [35]:
num_return_sequences = 10

# text = 'span {'# encoding the input text
text = 'import'# encoding the input text

input_ids = tokenizer.encode(text, return_tensors='pt')# getting out output
beam_output = model.generate(
    input_ids,
    do_sample=True, 
    max_length=n_positions, 
    top_k=10, 
    top_p=0.95, temperature=1.0,
    num_return_sequences=num_return_sequences,
    eos_token_id=1#tokenizer.get_vocab()['}']
)

for i  in range(num_return_sequences):
    print('\033[94m' + 'Prediction ' + str(i) + '\033[0m')
    s  = tokenizer.decode(beam_output[i])
    s = re.sub(r'<pad>', '', s)
    s = re.sub(text,'',s)
    print('\033[94m' + text + '\033[0m', end ="")
    print(s)

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Prediction 0
import { IContext } from "./blueprintjs/core';
Prediction 1
import _Action-ui/icons/styles/';
Prediction 2
import { Button } from '../../sharedUiComponents/common/styles';
Prediction 3
import * as React from'react';
Prediction 4
import _onsIcon from '@material-ui/core/svg';
Prediction 5
import { ables, DD_MM_PTSTP_ERSER_MM_PPS_SERPI_SAOTSSER_OO__A_PO_OTOPSIMAER_CO_
Prediction 6
import { useComponent, FanalIcon } from './common'
Prediction 7
import {
Prediction 8
import { useInput } from'material-ui/core/styles';
Prediction 9
import React from "react";
